In [1]:
import scipy.io as spio
import numpy as np

In [2]:
X = spio.loadmat(file_name='../data/bci/x.mat')

In [3]:
Y = spio.loadmat(file_name='../data/bci/y.mat')

In [4]:
Data = X['Intensification_Data']
SType = Y['Intensification_SType'].mean(axis=1)

In [21]:
# Normalizing input data
def normalize(inputs):
    return (inputs - inputs.mean(axis=0)[None,:,:]) / inputs.std(axis=0)[None,:,:]

# onehot vectorizing output labels
def one_hot(labels, n_class):
    """ One-hot encoding """
    expansion = np.eye(n_class)
    y = expansion[:, labels-1].T
    assert y.shape[1] == n_class, "Wrong number of labels!"

    return y

# get minibatches for learning
def get_batches(X, y, batch_size):
    """ Return a generator for batches """
    n_batches = len(X) // batch_size
    X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

    # Loop over batches and yield
    for b in range(0, len(X), batch_size):
        yield X[b:b+batch_size], y[b:b+batch_size]

In [22]:
# Standardize/normalize train and test
X_norm = normalize(inputs=Data)

In [23]:
# Deviding the input data into train and validation
# 30 % of data is test and the rest train
length = int(X_norm.shape[0] * 0.3)
X_train_norm = X_norm[: -length]
X_valid_norm = X_norm[-length:]

In [24]:
Y_train = np.array(SType[ :-length], dtype=int)
Y_valid = np.array(SType[-length: ], dtype=int)

### Hyperparameters

In [25]:
# Input data
# X_(5832, 192, 64): (n_trials, seq_len, n_channels), batch_size== n_trials
batch_size = X_train_norm.shape[0]// 100 # Batch size: number of trials
seq_len = X_train_norm.shape[1]          # Number of steps: each trial length
n_channels = X_train_norm.shape[2] # number of channels in each trial

# Output labels
# Y_(5832, 2): (n_trails, n_classes)
n_classes = Y_train.max() + 1

# Tweekable parameters
learning_rate = 0.001 #1e-3
epochs = 10 # num iterations for updating model
keep_prob = 0.95 # 90% neurons are kept and 10% are dropped out

In [26]:
Y_train_onehot = one_hot(labels=Y_train, n_class=n_classes)
Y_valid_onehot = one_hot(labels=Y_valid, n_class=n_classes)

In [27]:
(Y_train_onehot.shape, Y_valid_onehot.shape, X_train_norm.shape, X_valid_norm.shape)

((4536, 2), (1944, 2), (4536, 192, 64), (1944, 192, 64))

# GPUs availability or CPU

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [29]:
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs_')
labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels_')
keep_prob_ = tf.placeholder(tf.float32, name = 'keep_prob_')
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate_')

# Convolutional Layers, FC Layers, and Output layer

In [30]:
# (batch, 192, 9) --> (batch, 91, 18)
out_conv = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=12, strides=2, 
                         padding='valid', activation = tf.nn.relu)
print(out_conv.shape)

# (batch, 91, 18) --> (batch, 91*18) --> (batch, 91*18*2) 
in_fc = tf.reshape(out_conv, (-1, 91*18))
out_fc = tf.layers.dense(inputs=in_fc, units=91*18*2, activation=tf.nn.relu)
out_fc = tf.nn.dropout(x=out_fc, keep_prob=keep_prob_)
print(in_fc.shape, out_fc.shape)

# (batch, 91*18*2) --> (batch, 2) 
logits = tf.layers.dense(inputs=out_fc, units=n_classes)
print(logits.shape)

(?, 91, 18)
(?, 1638) (?, 3276)
(?, 2)


In [31]:
# Cost function and optimizer
cost_tensor = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_)
cost = tf.reduce_mean(input_tensor=cost_tensor)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [35]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over training minibatches
        for x, y in get_batches(X_train_norm, Y_train_onehot, batch_size):
            
            # Feed dictionary
            feed = {inputs_:x, labels_:y, keep_prob_:keep_prob, learning_rate_:learning_rate}
            train_loss, _ , train_acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            
            # Feed dictionary: no dropout/keep_prob; no learning rate/ no learning
            feed = {inputs_:X_valid_norm, labels_:Y_valid_onehot, keep_prob_:1.0}
            valid_loss, valid_acc = sess.run([cost, accuracy], feed_dict = feed)

            # Print at each iter/ epoch
            print("Epoch: {}/{}".format(e, epochs),
                  "Train loss: {:6f}".format(train_loss),
                  "Valid loss: {:.6f}".format(valid_loss),
                  "train acc: {:.6f}".format(train_acc),
                  "Valid acc: {:.6f}".format(valid_acc))

Epoch: 0/1 Train loss: 3.728856 Valid loss: 3.882883 train acc: 0.177778 Valid acc: 0.833333
Epoch: 0/1 Train loss: 5.595199 Valid loss: 4.940581 train acc: 0.844445 Valid acc: 0.833333
Epoch: 0/1 Train loss: 6.927114 Valid loss: 4.965527 train acc: 0.844445 Valid acc: 0.833333
Epoch: 0/1 Train loss: 5.876118 Valid loss: 4.693816 train acc: 0.822222 Valid acc: 0.830761
Epoch: 0/1 Train loss: 4.833348 Valid loss: 4.345282 train acc: 0.844445 Valid acc: 0.775206
Epoch: 0/1 Train loss: 1.654185 Valid loss: 3.951766 train acc: 0.733333 Valid acc: 0.732510
Epoch: 0/1 Train loss: 6.143596 Valid loss: 2.222680 train acc: 0.600000 Valid acc: 0.793210
Epoch: 0/1 Train loss: 1.509249 Valid loss: 1.308269 train acc: 0.866667 Valid acc: 0.735082
Epoch: 0/1 Train loss: 1.129480 Valid loss: 3.069188 train acc: 0.777778 Valid acc: 0.541152
Epoch: 0/1 Train loss: 7.246019 Valid loss: 1.442941 train acc: 0.355556 Valid acc: 0.727881
Epoch: 0/1 Train loss: 2.480733 Valid loss: 2.002934 train acc: 0.6000

Epoch: 0/1 Train loss: 0.535046 Valid loss: 0.495742 train acc: 0.800000 Valid acc: 0.824074
Epoch: 0/1 Train loss: 0.513361 Valid loss: 0.507823 train acc: 0.777778 Valid acc: 0.834362
Epoch: 0/1 Train loss: 0.485488 Valid loss: 0.538926 train acc: 0.822222 Valid acc: 0.834362
Epoch: 0/1 Train loss: 0.716833 Valid loss: 0.546976 train acc: 0.844445 Valid acc: 0.833333
Epoch: 0/1 Train loss: 1.045510 Valid loss: 0.508819 train acc: 0.822222 Valid acc: 0.834362
Epoch: 0/1 Train loss: 0.536986 Valid loss: 0.488827 train acc: 0.844445 Valid acc: 0.834362
Epoch: 0/1 Train loss: 0.488438 Valid loss: 0.495836 train acc: 0.800000 Valid acc: 0.826132
Epoch: 0/1 Train loss: 0.497003 Valid loss: 0.510209 train acc: 0.822222 Valid acc: 0.806584
Epoch: 0/1 Train loss: 0.559378 Valid loss: 0.514615 train acc: 0.755556 Valid acc: 0.802984
Epoch: 0/1 Train loss: 0.544965 Valid loss: 0.507759 train acc: 0.822222 Valid acc: 0.821502
Epoch: 0/1 Train loss: 0.584494 Valid loss: 0.491678 train acc: 0.8222